In [2]:
inputPath = "/data/students/bigdata_internet/lab2/word_frequency.tsv"
outputPath = "Lab2_Results/"

In [3]:
rdd = sc.textFile(inputPath)
new_rdd = rdd.map(lambda x: x.split('\t'))
final_rdd = new_rdd.map(lambda x: (x[0], int(x[1])))
final_rdd.take(5)
final_rdd.top(5,lambda x: x[1])

339819

- reading the file from that path
- splitting the each item using '\t'
- csting the second parth which the frequency to integer
- taking 5 samples from rdd
- picking the first most frequest words

there are 339819 words in total

### 1.0.1
drawing 5 samples from RDD using take() function

### 1.0.2
for picking the top nth row we would use top() function which the second parameter is the function for sorting

In [3]:
final_rdd.count()

339819

### 1.0.3
- using the count method on rdd we get the total number of words which is 339819
- if there are repeatitive words we can use distinct() function

### 1.0.4
it is a file. A TSV file is a tab-separated values file commonly used by spreadsheet applications to exchange data between databases

In [4]:
prefix = 'ho'

In [5]:
fltr_rdd = final_rdd.filter(lambda x : x[0].startswith(prefix))
fltr_rdd.count()
fltr_rdd.top(1, lambda x: x[1])

[('how', 36264)]

### 1.1.1
there are 1519 words which start with 'ho' as prefix

### 1.1.2
top most frequest word starting with 'ho' is [('how', 36264)]

- first we filter the original RDD to get the words starting with 'ho'
- use top() method to get tghe most frequent then get the first item of the rdd then get the second field of the array

In [6]:
fltr_rdd = final_rdd.filter(lambda x : x[0].startswith(prefix))
print('max frequent: ', fltr_rdd.top(1, lambda x: x[1])[0][1])

[Stage 5:=============================>                             (1 + 1) / 2]

max frequent:  36264


### 1.1.2
second method

- first we filter the original RDD to get the words starting with 'ho'
- sort the resulted rdd using takeordered() function which the second part is the method for sorting
- printing the second filed of the array of the first row of the list

In [7]:
fltr_rdd = final_rdd.filter(lambda x : x[0].startswith(prefix))
max_freq = fltr_rdd.takeOrdered(1, lambda n: -1*n[1])
print(sorted_rdd[0][1])

NameError: name 'sorted_rdd' is not defined

### 1.1.2
third method

- first we filter the original RDD to get the words starting with 'ho'
- sorting the rdd using sortBy() method according to the second field of array
- printing the second field of the first array of the first resulted rdd picked by take()

In [8]:
fltr_rdd = final_rdd.filter(lambda x : x[0].startswith(prefix))
sortedPairs = fltr_rdd.sortBy(lambda pair: pair[1], False)
max_freq = (sortedPairs.take(1))[0][1]
print(max_freq)

[Stage 9:=============================>                             (1 + 1) / 2]

36264


## 1.3
## filtering the top 70% most frequent

### 1.3.1

counting the number of the words that their frequency is >= 70% of the max_freq

In [9]:
max_70freq_rdd = fltr_rdd.filter(lambda el: el[1] > max_freq*0.7)
max70count = max_70freq_rdd.count()
print(max70count)

[Stage 11:=============================>                            (1 + 1) / 2]

2


1.3.2

saving the words the hdfs

- mapping each line of the rdd to change it to the desired style word;
- path to save in HDFS
- saving the resulted RDD

In [11]:
final_words = max_70freq_rdd.map(lambda x: x[0]+';')
max_filtered_path = 'lab2_70_max_freq_words/'
# final_words.collect()
final_words.saveAsTextFile(max_filtered_path)

# 2
## python script
### script used in the file is as shown:
- almost every line is as the same as before but with a small change in the last line
- mapping all each line and changing the style to the desired one word - frequency,

<code>
import os
import sys
from pyspark import SparkConf, SparkContext
    
conf = SparkConf().setAppName("My app")
sc = SparkContext(conf = conf)
inputPath = "/data/students/bigdata_internet/lab2/word_frequency.tsv"
outputFolderName = str(sys.argv[1]) + '/'
prefixStr = str(sys.argv[2])
rdd = sc.textFile(inputPath)
new_rdd = rdd.map(lambda x: x.split('\t'))
final_rdd = new_rdd.map(lambda x: (x[0], int(x[1])))
fltr_rdd = final_rdd.filter(lambda x : x[0].startswith(prefixStr))
changed_rdd = fltr_rdd.map(lambda x: (x[0] + ' - ' + str(x[1]) +','))
changed_rdd.saveAsTextFile(outputFolderName)
</code>

### 2.1
spark-submit --master local --deploy-mode client 'lab2_prefix_script.py' lab2scriptfolder ho

In [27]:
!spark-submit --master local --deploy-mode client 'lab2_prefix_script.py' lab22scriptt ho

22/12/20 00:04:20 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/20 00:04:20 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
Program took 4.645367622375488 seconds to run                                   


the command shown above execution time is :
- Program took 4.714531421661377 seconds to run
- Program took 4.645367622375488 seconds to run 

In [30]:
!spark-submit --master yarn --deploy-mode client 'lab2_prefix_script.py' lab2scriptttt ho

22/12/20 00:07:13 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/20 00:07:13 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/20 00:07:22 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!
Traceback (most recent call last):
  File "/home/students/s308885/lab2_prefix_script.py", line 22, in <module>
    changed_rdd.saveAsTextFile(outputFolderName)
  File "/opt/cloudera/parcels/CDH-6.2.1-1.cdh6.2.1.p0.1425774/lib/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 1570, in saveAsTextFile
  File "/opt/cloudera/parcels/CDH-6.2.1-1.cdh6.2.1.p0.1425774/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
  File "/opt/cloudera/parcels/CDH-6.2.1-1.cdh6.2.1.p0.1425774/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 328, in get_return_value
py4j.protocol.Py4JJavaError: An error occurred while c

the commad above was ran couple of times and there was different durations:
- Program took 24.257280588150024 seconds to run
- Program took 25.14168953895569 seconds to run

# Bonus Task

In [4]:
originPath = '/data/students/bigdata_internet/lab2/finefoods_text.txt'

def splittor(reading):
    fields = reading.split(" ")
    return fields


origin_rdd = sc.textFile(originPath)
seperated_rdd = origin_rdd.flatMap(splittor)
seperated_rdd.count()

51967093

### 3.1
#### total number of words with repeatition is : 51967093

- address for the original data
- function for the flatmap() method in order to seperate words of a string of each line
- reading the file
- using the flatMap() to get a single RDD of the words of eachline, if we use map then it would give us arrays of words of each line
- counting the words

## 3.2 !